# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#import independecies
import os 
import pandas as pd 
import requests
import json
from api_keys import weather_api_key
from api_keys import g_key
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress



## Generate Cities List

In [2]:
#finding files of csv file where we saved the world cities 
filepath="../Resources/worldcities.csv"
worldcities=pd.read_csv(filepath)

#convering the city_ascii into a list 
city_list=worldcities['city_ascii'].tolist()

#getting city names 
cities=city_list[0:600]
cities


#using iloc getting the top 600 rows and city_ascii column (unique city name) of worldcities df to create a new city df 
#cities_df=pd.DataFrame(worldcities.iloc[0:600,1])
#rename city_ascii as city which make more sens 
#weather=cities_df.rename(columns={"city_ascii": "city"})
#weather.head()

['Tokyo',
 'Jakarta',
 'Delhi',
 'Mumbai',
 'Manila',
 'Shanghai',
 'Sao Paulo',
 'Seoul',
 'Mexico City',
 'Guangzhou',
 'Beijing',
 'Cairo',
 'New York',
 'Kolkata',
 'Moscow',
 'Bangkok',
 'Buenos Aires',
 'Shenzhen',
 'Dhaka',
 'Lagos',
 'Istanbul',
 'Osaka',
 'Karachi',
 'Bangalore',
 'Tehran',
 'Kinshasa',
 'Ho Chi Minh City',
 'Los Angeles',
 'Rio de Janeiro',
 'Nanyang',
 'Baoding',
 'Chennai',
 'Chengdu',
 'Lahore',
 'Paris',
 'London',
 'Linyi',
 'Tianjin',
 'Shijiazhuang',
 'Zhoukou',
 'Lima',
 'Hyderabad',
 'Handan',
 'Bogota',
 'Weifang',
 'Nagoya',
 'Wuhan',
 'Heze',
 'Ganzhou',
 'Tongshan',
 'Chicago',
 'Luanda',
 'Changsha',
 'Fuyang',
 'Kuala Lumpur',
 'Jining',
 'Dongguan',
 'Jinan',
 'Foshan',
 'Hanoi',
 'Pune',
 'Chongqing',
 'Changchun',
 'Zhumadian',
 'Ningbo',
 'Cangzhou',
 'Nanjing',
 'Hefei',
 'Ahmedabad',
 'Hong Kong',
 'Zhanjiang',
 'Shaoyang',
 'Hengyang',
 'Khartoum',
 'Nantong',
 'Yancheng',
 'Nanning',
 "Xi'an",
 'Shenyang',
 'Tangshan',
 'Santiago',
 'Zh

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#creating params holding keys and cities, create url for google api 
base_url_geocode="https://maps.googleapis.com/maps/api/geocode/json"

#creating partial query url for open weather 
base_url_weather = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url_weather = f"{base_url_weather}appid={weather_api_key}&units={units}&q="

#set up empty list to hold response info 
lat=[]
lng=[]
max_temp=[]
humidity=[]
cloudiness=[]
wind_speed=[]
country=[]
datetime=[]
# Loop through the cities and get lat, lng from google api 
#loop through cities and get weather data from open weather api 
for city in cities:
    #getting json format geo data and etract lat & lng 
    params = {"key": g_key,"address":city}
    geo=requests.get(base_url_geocode,params=params).json()
    try:
        lat.append(geo["results"][0]["geometry"]["location"]["lat"])
        lng.append(geo["results"][0]["geometry"]["location"]["lng"])
    except:
        pass
    #getting json format weather data and etract relevant data 
   
    weather=requests.get(query_url_weather + city).json()
    try:
        max_temp.append(weather['main']['temp_max'])
        humidity.append(weather['main']['humidity'])
        cloudiness.append(weather['clouds']['all'])
        wind_speed.append(weather['wind']['speed'])
        country.append(weather['sys']['country'])
        datetime.append(weather['dt'])
    except:
        pass
#create dataframe to hold above results
weather_dict = {
    "city": cities,
    "lat": lat,
    "lng": lng,
    "Max Temp": max_temp,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed":  wind_speed,
    "Country": country,
    "Date Time":datetime
}

weather_df=pd.DataFrame(weather_dict)
weather_df.to_csv("weather_data.csv")
weather_df.head()
 


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


#it is easier to just to get all rows of humidity less than 100, and getting clean df (istead of drop)
clean_city_data=weather_df[weather_df.Humidity<=100]
clean_city_data.head()


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
clean_city_data.plot.scatter(x='lat',y='Max Temp',c='DarkBlue')
plt.title("Latitude vs. Temperature Plot")
plt.xlabel("Latitude")
plt.ylabel("Temperature")
plt.show

## Latitude vs. Humidity Plot

In [ ]:
clean_city_data.plot.scatter(x='lat',y='Humidity',c='DarkBlue')
plt.title("Latitude vs. Humidity Plot")
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.show

## Latitude vs. Cloudiness Plot

In [ ]:
clean_city_data.plot.scatter(x='lat',y='Cloudiness',c='DarkBlue')
plt.title("Latitude vs. Cloudiness Plot")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.show

## Latitude vs. Wind Speed Plot

In [ ]:
clean_city_data.plot.scatter(x='lat',y='Wind Speed',c='DarkBlue')
plt.title("Latitude vs. Wind Speed Plot")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.show

## Linear Regression

In [ ]:
northern=clean_city_data[clean_city_data.lat>=0]
southern=clean_city_data[clean_city_data.lat<0]


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x=northern['lat']
y=northern['Max Temp']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(20,80),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Max Temp)")
plt.title('Northern Hemisphere - Max Temp vs. Latitude Linear Regression')
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x=southern['lat']
y=southern['Max Temp']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(-15,80),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Max Temp)")
plt.title('Southern Hemisphere - Max Temp vs. Latitude Linear Regression¶')
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x=northern['lat']
y=northern['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(25,80),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Humidity)")
plt.title('Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression')
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x=southern['lat']
y=southern['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(-15,70),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Humidity)")
plt.title('Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression')
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x=northern['lat']
y=northern['Cloudiness']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(25,70),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Cloudiness)")
plt.title('Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression')
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x=southern['lat']
y=southern['Cloudiness']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(-15,30),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Cloudiness)")
plt.title('Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression')
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x=northern['lat']
y=northern['Wind Speed']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(20,7),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Wind Speed)")
plt.title('Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression')
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x=southern['lat']
y=southern['Wind Speed']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")

# Annotate linear regression
plt.annotate(line_eq,(-15,7),fontsize=15,color="blue")

# Add labels and title to plot
plt.xlabel("Latitude ")
plt.ylabel("Wind Speed")
plt.title('Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression')
plt.show()